In [ ]:
import torch
import pickle

from encoding_utils import *
from flatten_lattice import *
from model_construct import *

In [ ]:
def submask_help(words):
    msk = []
    for i in range(len(words)-1):
        if "#" in words[i+1]:
            msk.append(0)
        else:
            msk.append(1)
    msk.append(1)
    return torch.tensor(msk)

def subword_mask_all ():
    msk = torch.ones_like(sents)
    for i in range(0, len(sents)):
        tmp = [tok.decode(s) for s in sents[i]]
        msk[i] = submask_help(tmp)
    return msk

def get_acc():
    # simplify prediction tensors
    ysimp = torch.clone(ylabels)
    psimp = torch.argmax(pred1, dim=2)
    # clean up labels
    sm = subword_mask_all()
    ysimp[sents==0] = 0
    ysimp[sents==102] = 0
    ysimp[sm==0] = 0
    ysimp[:, 0] = 0
    # apply cleanaup to x 
    psimp[ysimp==0] = 0
    # apply cleanaup to x 
    psimp[ysimp==0] = 0
    # compute accuracy
    acc = 1 - ((ysimp-psimp).count_nonzero())/ysimp.count_nonzero()
    return acc, ysimp, psimp

def get_tmap_acc(tmaps, sents):
    a, ysmp, p = get_acc()
    assert len(tmaps)==len(ysmp)
    cor = 0
    tot = 0
    for a in ysmp.nonzero():
        if int(p[a[0], a[1]]) in tmaps[a[0]][str(int(sents[a[0], a[1]]))]:
            #print(int(ysmp[a[0], a[1]]), " ",  tmaps[a[0]][str(int(sents[a[0], a[1]]))])
            cor+=1
        else:
            print(int(ysmp[a[0], a[1]]))
            print(int(p[a[0], a[1]]), " ",tmaps[a[0]][str(int(sents[a[0], a[1]]))] )
        tot+=1
    print(cor/tot)
    print(cor)
    print(tot)
    return cor/tot
            

def get_err_tensors():
    diff = ysimp-psimp
    diff = diff.abs().bool().int()
    errg = ysimp
    errp = psimp
    errg[diff==0] = 0
    errp[diff==0] = 0
    return errg, errp


def print_decoded(toks):
    nt = []
    for t in toks:
        if t==100 or t==101 or t==102 or t==0:
            continue
        nt.append(t)
    print(tok.decode(nt))
    return tok.decode(nt)
    
# 
def error_vis(ind):
    nt = print_decoded(sents[ind])
    p = [lablist[int(errp[ind][i])] for i in errp[ind].nonzero()]
    g = [lablist[int(errg[ind][i])] for i in errp[ind].nonzero()]
    s = [tok.decode(int(sents[ind][i])) for i in errp[ind].nonzero()]
    print("ERRORS : ")
    print("(predicted, gold, token)")
    print(list(zip(p, g, s)))
    return nt
    
def vis_model_pred(strinp):
    tokinps = tok(strinp, return_tensors='pt').input_ids.to(device)
    msk = torch.tril(torch.ones((len(tokinps[0]), len(tokinps[0]))))
    preds = bmodel(tokinps, attmasks=torch.stack([msk]).to(device))
    preds = torch.argmax(preds, dim=2)
    labs = [lablist[int(p)] for p in preds[0]]
    tinps = [tok.decode(int(t))for t in tokinps[0]]
    #print(list(zip(labs, tinps)))

In [ ]:
accuracy, ysimp, psimp = get_acc()


In [ ]:
len(ysimp.nonzero())

In [ ]:
tok = fl.bert_tok

# loads in odata with keys: tmaps, masks, pgraphs
with open('torchsaved/outputv7.pkl', 'rb') as file:
    odata = pickle.load(file)
    
with open('./a3distrib/lab_vocab.json') as json_file:
    labels = json.load(json_file)
    
# load in other metadata
bmodel = load_model(labels)
sents, posids = create_inputs(odata['pgraphs'])

In [ ]:
ylabels = tmap_pos_goldlabels(odata['tmaps'], sents)   
lablist = [l for l in labels.keys()]
torch.cuda.empty_cache()

pg = odata['pgraphs']

In [ ]:
sents, posids = create_inputs(odata['pgraphs'])
pred1 = bmodel(sents, mod_posids(posids), odata['masks'])
accuracy, ysimp, psimp = get_acc()

print("accuracy: ", accuracy)

errg, errp = get_err_tensors()

In [ ]:
ysimp[0]

In [ ]:
# can put any number between 0-100 and it will show an example and errors with the example
INP = 9
for i in range(0, 100):
    vis_model_pred(error_vis(i))

In [ ]:
len((ysimp-psimp).nonzero())

In [ ]:
len(ysimp.nonzero())

In [ ]:
#################### BEYOND THIS IS RANDOM SANITY CHECKING ########################################

In [ ]:
ta = get_tmap_acc(odata['tmaps'], sents)

In [30]:
accuracy

tensor(0.9391, device='cuda:3')

In [ ]:
goldlist = []
predlist = []
toklist = []
for e in errg.nonzero():
    goldlist.append(lablist[int(errg[e[0], e[1]])])
    predlist.append(lablist[int(errp[e[0], e[1]])])
    toklist.append(tok.decode(int(sents[e[0], e[1]])))
    
zipped = list(zip(goldlist, predlist, toklist))
zipcln = []
cnt = 0
for z in zipped:
    if z[1]==z[2]:
        cnt+=1
        continue
    zipcln.append(z)
    

In [ ]:
# sanity check masking 
sents.shape

In [ ]:
odata['masks'].shape

In [ ]:
def print_mask_leadup(ind):
    msk = odata['masks'][ind]
    smat = torch.zeros_like(msk)
    smat[:, :] = sents[ind]
    assert msk.shape == smat.shape
    return smat, msk

In [ ]:
sm, m = print_mask_leadup(0)

In [ ]:
sm[m==0]=0

In [ ]:
inps = tok.batch_decode(sm.int())

In [ ]:
inps = [txt.replace(" [PAD]", "") for txt in inps]

In [ ]:
posids[0]

In [ ]:
a = torch.zeros(5, 5)
a[:3, :3] = torch.ones(3, 3)

In [ ]:
torch.tril(a)